In [1]:
import numpy as np
import torch
from torch.utils import data # 和normal不同的地方
from d2l import torch as d2l

torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = torch.tensor(4.2)
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [10]:
def load_array(data_arrays, batch_size, is_train=True):
    # 构造一个pytorch迭代器
    dataset = data.TensorDataset(*data_arrays)
    '''
    dataset里面有两个element，一个是X，一个是y
    X的shape为(1000, true_w.shape[0])，每一行是一个example，总共1000行
    y的shape为(1000, 1)，每一行是一个example，总共1000行
    下面的dataloader主要是做了个顺序打乱工作，顺便根据batch_size返回对应数目的mini-batch
    '''
    return data.DataLoader(dataset, batch_size, shuffle=is_train, generator=torch.Generator(device='cuda')) # 新版pytorch记得在dataloader后指定generator的device

batch_size = 10
data_iter = load_array((features, labels), batch_size)


(tensor([ 0.7869, -0.1846]), tensor([6.4129]))


In [4]:
from torch import nn
'''
nn.Linear(m, n)定义了一个线性回归模型，第一层有m个unit，第二层有n个unit
nn.Sequential里面的参数是把一堆model按顺序叠在一起
'''
net = nn.Sequential(nn.Linear(2, 1))
net[0].weight.data.normal_(0, 0.01) # 为线性模型的weight和bias初始化参数
net[0].bias.data.fill_(0)

loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [8]:
num_epoch = 3
for epoch in range(num_epoch):
    for X, y in data_iter:
        '''
        1.拿出一个小批量的数据（注意这里的X是矩阵，y是向量，计算loss
        2.清零梯度
        3.根据loss反向传播一次（这一步之后就求出了导数）
        4.迭代一次
        5.重复上面步骤直到拿完所有数据
        '''
        l = loss(net(X),  y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print('epoch @{} loss : {}'.format(epoch + 1, float(l)))

epoch @1 loss : 9.277692879550159e-05
epoch @2 loss : 9.209680138155818e-05
epoch @3 loss : 9.198694169754162e-05
